In [3]:
!pip3 install --upgrade ibm-watson
!pip3 install --upgrade watson-developer-cloud


Requirement already up-to-date: ibm-watson in /usr/local/lib/python3.7/site-packages (4.5.0)
Requirement already up-to-date: watson-developer-cloud in /usr/local/lib/python3.7/site-packages (2.10.1)


In [6]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(),'..'))
import json
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [76]:
url = "Your url. Copy from IBM cloud"
authenticator = IAMAuthenticator('Your apikey. Copy from IBM cloud')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator)
language_translator.set_service_url(url)

In [88]:
#간단한 번역 테스트
translation = language_translator.translate(
    text='Hello', model_id='en-ko').get_result()
print(json.dumps(translation, indent=2, ensure_ascii=False))

SyntaxError: invalid syntax (<ipython-input-88-b6c3b055c371>, line 1)

In [90]:
#서버로 번역할 파일 보내기
with open('./en.txt', 'rb') as file:
    result = language_translator.translate_document(
        file=file,
        file_content_type='text/plain',
        filename='en.txt',
        model_id='en-ko').get_result()
    document_ID = result['document_id']
    print(json.dumps(result, indent=2))


{
  "document_id": "fdea368a-cea8-4c34-b1b4-c27fc09084a3",
  "filename": "en.txt",
  "model_id": "en-ko",
  "source": "en",
  "target": "ko",
  "status": "processing",
  "created": "2020-07-20T02:29:17Z"
}


In [91]:
#번역 상태 확인
result = language_translator.get_document_status(
    document_id=document_ID).get_result()
print(json.dumps(result, indent=2))


{
  "document_id": "fdea368a-cea8-4c34-b1b4-c27fc09084a3",
  "filename": "en.txt",
  "model_id": "en-ko",
  "source": "en",
  "target": "ko",
  "status": "available",
  "created": "2020-07-20T02:29:17Z",
  "completed": "2020-07-20T02:29:18Z",
  "word_count": 1182,
  "character_count": 6686
}


In [92]:
#번역한 내용을 저장하기
with open('./translated.txt', 'wb') as f:
    result = language_translator.get_translated_document(
        document_id=document_ID,
        accept='text/plain').get_result()
    f.write(result.content)

In [94]:
#서버에 남아있는 번역 결과물 삭제하기
language_translator.delete_document(document_id=document_ID)